# QUERY 2

## SQL implementation

In [1]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1805,application_1765289937462_1789,pyspark,idle,Link,Link,None,
1807,application_1765289937462_1791,pyspark,busy,Link,Link,None,
1808,application_1765289937462_1792,pyspark,idle,Link,Link,None,
1809,None,pyspark,starting,,,None,


In [2]:
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, DoubleType, StringType

# Define the data schemas
crime_data_schema = StructType([
    StructField("DR_NO", IntegerType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", IntegerType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", IntegerType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", IntegerType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", DoubleType()),
    StructField("LON", DoubleType()),
])

re_codes_schema = StructType([
    StructField("Vict Descent", StringType()),
    StructField("Vict Descent Full", StringType()),
])

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1810,application_1765289937462_1794,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import SparkSession
import time

# Starting the timer
start_time = time.time()

# Loading data from the 2 crime datasets and combining them
crime_data_1 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv",
    header=True,
    schema=crime_data_schema
)

crime_data_2 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv",
    header=True,
    schema=crime_data_schema
)

crime_data_df = crime_data_1.union(crime_data_2)

# Loading the RE codes
re_codes_df = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/RE_codes.csv",
    header=True,
    schema=re_codes_schema
)

# Creating the SQL View "crimes" and "re_codes" from the corresponding dataframes
crime_data_df.createOrReplaceTempView("crimes")
re_codes_df.createOrReplaceTempView("re_codes")

# Query 0: Adding the filed year (of occurence) to the table
# After every query we save the resulting dataframe in an SQL View!
query0 = """
            SELECT *, year(to_timestamp(`DATE OCC`, 'yyyy MMM dd hh:mm:ss a')) AS year
            FROM crimes;
         """
crime_data_query0_df = spark.sql(query0)
crime_data_query0_df.createOrReplaceTempView("crimes")

# Query 1: Grouping data by year and ethnicity
# We exclude null data
query1 = """
            SELECT year, `Vict Descent`, COUNT(*) AS Total FROM crimes 
            WHERE year IS NOT NULL AND `Vict Descent` IS NOT NULL 
            GROUP BY year, `Vict Descent`
         """
crime_data_query1_df = spark.sql(query1)
crime_data_query1_df.createOrReplaceTempView("crimes_grouped")

# Query 2: Partition previous result over years, then sort, take the top 3 and calculate percentage
query2 = """
            SELECT year, `Vict Descent`, Total AS `#`, ROUND(100*(Total/sum),2) as `%` FROM (
                SELECT *, ROW_NUMBER() OVER (
                    PARTITION BY year
                    ORDER BY Total DESC
                ) AS rn, SUM(Total) OVER (PARTITION BY year) as sum
                FROM crimes_grouped
            ) AS t
            WHERE rn <= 3;
         """
crime_data_query2_df = spark.sql(query2)
crime_data_query2_df.createOrReplaceTempView("crimes_grouped")

# Query 3: Joining the "crimes_grouped" with the "re_codes" view
# in order to have the Full Victim Descent instead of the abbreviated letter
# and ordering by year and total number of incidents
query3 = """
            SELECT year, `Vict Descent Full` AS `Victim Descent`, `#`, `%`
            FROM crimes_grouped NATURAL JOIN re_codes
            ORDER BY year DESC, `#` DESC
         """
result_df = spark.sql(query3)
result_df.show(1000, truncate=False)

end_time = time.time()
print(f"Execution Time: {end_time - start_time} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+-----+-----+
|year|Victim Descent        |#    |%    |
+----+----------------------+-----+-----+
|2025|Hispanic/Latin/Mexican|34   |40.48|
|2025|Unknown               |24   |28.57|
|2025|White                 |13   |15.48|
|2024|Hispanic/Latin/Mexican|28576|29.05|
|2024|White                 |22958|23.34|
|2024|Unknown               |19984|20.32|
|2023|Hispanic/Latin/Mexican|69401|34.55|
|2023|White                 |44615|22.21|
|2023|Black                 |30504|15.19|
|2022|Hispanic/Latin/Mexican|73111|35.64|
|2022|White                 |46695|22.76|
|2022|Black                 |34634|16.88|
|2021|Hispanic/Latin/Mexican|63676|35.08|
|2021|White                 |44523|24.53|
|2021|Black                 |30173|16.62|
|2020|Hispanic/Latin/Mexican|61606|35.33|
|2020|White                 |42638|24.45|
|2020|Black                 |28785|16.51|
|2019|Hispanic/Latin/Mexican|72458|36.38|
|2019|White                 |48863|24.54|
|2019|Black                 |33157

## DataFrame implementation

In [4]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1811,application_1765289937462_1795,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1805,application_1765289937462_1789,pyspark,idle,Link,Link,None,
1807,application_1765289937462_1791,pyspark,idle,Link,Link,None,
1808,application_1765289937462_1792,pyspark,idle,Link,Link,None,
1811,application_1765289937462_1795,pyspark,idle,Link,Link,None,✔
1812,application_1765289937462_1796,pyspark,starting,Link,Link,None,


In [5]:
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, DoubleType, StringType

# Define the data schemas
crime_data_schema = StructType([
    StructField("DR_NO", IntegerType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", IntegerType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", IntegerType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", IntegerType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", DoubleType()),
    StructField("LON", DoubleType()),
])

re_codes_schema = StructType([
    StructField("Vict Descent", StringType()),
    StructField("Vict Descent Full", StringType()),
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql.functions import col, row_number, desc, sum, round, year, to_timestamp
from pyspark.sql.window import Window
import time

# Starting the timer
start_time = time.time()

# Loading data from the 2 crime datasets and combining them
crime_data_1 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv",
    header=True,
    schema=crime_data_schema
)

crime_data_2 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv",
    header=True,
    schema=crime_data_schema
)

crime_data_df = crime_data_1.union(crime_data_2)

# Loading the RE codes
re_codes_df = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/RE_codes.csv",
    header=True,
    schema=re_codes_schema
)

# Extracting year of occurence and adding it to the dataframe
crime_data_df = crime_data_df.withColumn(
    "year",
    year(to_timestamp(col("DATE OCC"), "yyyy MMM dd hh:mm:ss a"))
)

# Grouping by year and ethnicity
# Then we remove the null values
crime_data_grouped_df = crime_data_df.groupBy(col("year"), col("Vict Descent")).count()
crime_data_grouped_df = crime_data_grouped_df.na.drop()

# Define window for ordered descent counts per year
rank_window = Window.partitionBy("year").orderBy(desc("count"))

# Define a window for total per year (no ordering needed)
total_window = Window.partitionBy("year")

# Add row_number and total per year
# Then, get the top 3 rows and calculate the percent
crime_data_ordered_df = crime_data_grouped_df.withColumn("rank", row_number().over(rank_window)) \
                            .withColumn("total_count", sum("count").over(total_window)) \
                            .filter(col("rank") <= 3) \
                            .withColumn("%", round(100 * col("count") / col("total_count"), 2)) \
                            .withColumnRenamed("count", "#") \
                            .drop("rank") \
                            .drop("total_count")

# Renaming
result_df = crime_data_ordered_df.join(re_codes_df, "Vict Descent") \
                .drop("Vict Descent") \
                .withColumnRenamed("Vict Descent Full", "Victim Descent") \
                .select("year", "Victim Descent", "#", "%") \
                .orderBy(desc("year"), desc("#"))

result_df.show(1000, truncate=False)
end_time = time.time()
print(f"Execution Time: {end_time - start_time} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+-----+-----+
|year|Victim Descent        |#    |%    |
+----+----------------------+-----+-----+
|2025|Hispanic/Latin/Mexican|34   |40.48|
|2025|Unknown               |24   |28.57|
|2025|White                 |13   |15.48|
|2024|Hispanic/Latin/Mexican|28576|29.05|
|2024|White                 |22958|23.34|
|2024|Unknown               |19984|20.32|
|2023|Hispanic/Latin/Mexican|69401|34.55|
|2023|White                 |44615|22.21|
|2023|Black                 |30504|15.19|
|2022|Hispanic/Latin/Mexican|73111|35.64|
|2022|White                 |46695|22.76|
|2022|Black                 |34634|16.88|
|2021|Hispanic/Latin/Mexican|63676|35.08|
|2021|White                 |44523|24.53|
|2021|Black                 |30173|16.62|
|2020|Hispanic/Latin/Mexican|61606|35.33|
|2020|White                 |42638|24.45|
|2020|Black                 |28785|16.51|
|2019|Hispanic/Latin/Mexican|72458|36.38|
|2019|White                 |48863|24.54|
|2019|Black                 |33157